In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from trace_IO import *
from OptimumFilter import *

fs = 3906250
sigma = 0.7674
thresholds = [round(n * sigma, 4) for n in [3, 4, 5]]
energy_values = list(range(5, 101, 5))
pass_rates_by_threshold = {th: [] for th in thresholds}
n_sets = 100

# Load templates and noise PSD
qp_template = np.load("templates/vac_ch_template.npy")
noise_psd = np.load("../../templates/noise_psd_from_MMC.npy")

vac_of = OptimumFilter(qp_template, noise_psd, fs)


In [4]:
energy = 5
trace_path = f"/ceph/dwong/trigger_samples/lts/traces_energy_{energy}.zst"
loaded_traces = load_traces_from_zstd(trace_path, n_traces=n_sets)
example = loaded_traces[91][0][:32768]

In [7]:
import time

start_time = time.time()
ampl = vac_of.fit(example)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")


Time taken: 0.001983 seconds


In [ ]:
energy = 5
trace_path = f"/ceph/dwong/trigger_samples/lts/traces_energy_{energy}.zst"
loaded_traces = load_traces_from_zstd(trace_path, n_traces=n_sets)

(250000,)


In [ ]:
example = loaded_traces[91][0]

window_size = 32768
num_windows = len(example) - window_size + 1

def fit_window(i):
    segment = example[i:i+window_size]
    return vac_of.fit(segment)


ampl_arr = []
chisq_arr = []
for i in range(num_windows):
    ampl = fit_window(i)
    ampl_arr.append(ampl)
ampl_arr = np.array(ampl_arr)


In [14]:
import numpy as np

# Parameters
window_size = 32768
num_traces = 9

# Infer shape from the first trace
trace_length = loaded_traces[0][0].shape[0]
num_windows = trace_length - window_size + 1

# Output array: (9 traces, num_windows per trace)
all_ampl_arr = np.empty((num_traces, num_windows), dtype=np.float32)

# Allocate a reusable buffer to hold each window
window = np.empty(window_size, dtype=loaded_traces[0][0].dtype)

# Loop over traces
for trace_idx in range(num_traces):
    trace = loaded_traces[trace_idx][0]  # 1D array

    # Direct reference to row in output for speed
    ampl_arr = all_ampl_arr[trace_idx]

    # Slide window across trace
    for i in range(num_windows):
        # Copy segment to buffer instead of slicing
        window[:] = trace[i:i+window_size]
        ampl_arr[i] = vac_of.fit(window)


In [17]:
all_ampl_arr.shape

(9, 217233)